## Исследование надёжности заёмщиков

Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

### Шаг 1. Откройте файл с данными и изучите общую информацию. 

In [3]:
import pandas as pd
df = pd.read_csv('/datasets/data.csv')
df.info()
df.isnull().sum()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


 Вывод

Значения NaN в столбцах days_employed и total_income, возможно данные были выгружены некорректно.
В столбце total_income есть отрицательные значения

### Шаг 2. Предобработка данных

### Обработка пропусков

In [2]:
#создание нового датасета для работы
df_borrower = pd.DataFrame()
df_borrower = df
df_borrower = df_borrower.fillna(0)

df_group = df_borrower.groupby('income_type')['total_income'].median()

income_type_unique = df_borrower['income_type'].sort_values().unique()

ratio_list = []
median_list = []

#цикл для заполнения значений 0 медианой в зависимсти от типа занятости заямщика
for element in income_type_unique:
    income = df_borrower[df_borrower['income_type'] == element]['total_income'].replace(0,df_group[element]).median()
    type_count = df_borrower[df_borrower['income_type'] == element]['income_type'].count()
    debt_count = df_borrower[(df_borrower['income_type'] == element)&(df_borrower['debt'] == 0)]['income_type'].count()
    ratio = (debt_count/type_count)*100
    ratio_list.append(ratio)
    median_list.append(income)

#df_borrower.isnull().sum()


### Вывод

Количество NaN составляет 10% от общих данных, поэтому не удаляем, а заменяем в зависимости от типа занятости заемщика на медиану.

### Замена типа данных

In [145]:
df_borrower['total_income'] = df_borrower['total_income'].astype('int') 
df_borrower['days_employed'] = df_borrower['days_employed'].astype('int') 
df_borrower.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null int64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null int64
purpose             21525 non-null object
dtypes: int64(7), object(5)
memory usage: 2.0+ MB


### Вывод

Изменяем тип данных методом astype(), т.к. нужны целочисленные значения. 

### Обработка дубликатов

In [30]:
#проверка наличия дубликатов
print('Почему то дубликатов нет, опять что неправильно:)', df_borrower.duplicated().sum())

#замена некорректных значений количества детей
df_borrower['children'] = df_borrower['children'].replace(-1, 1)
df_borrower['children'] = df_borrower['children'].replace(20, 2)
df_borrower['children'].value_counts()

#убираем отрицательные значения
try: 
    adf_borrower['days_employed'] = df_borrower['days_employed'].abs()
except: 
    print('Проверьте значения в столбце days_employed')
df_borrower[df_borrower['days_employed'] < 0].shape[0] 

Почему то дубликатов нет, опять что неправильно:) 0
Проверьте значения в столбце days_employed


15906

### Вывод

Значение "-1" в столбце "количество детей"  - скорее всего человеческий фактор, замена на 1.
Отрицательные значения убираем используя метод abs() и конструкцию try-except для проверки.

### Лемматизация

In [147]:
from pymystem3 import Mystem
m = Mystem()

purpose_unique = df_borrower['purpose'].unique()
#новый DataFrame с уникальными целями кредитов
df_borrower_purpose = pd.DataFrame(data=purpose_unique)

#функция для лемматизации и классификации целей 
def purpose_group(purpose_unique): 
    lemmas = m.lemmatize(purpose_unique)
    print(lemmas)
    if 'жилье' in lemmas:
        return 1     
    if 'недвижимость' in lemmas:
        return 2
    if 'образование' in lemmas:
        return 3
    if 'автомобиль' in lemmas:
        return 4
    if 'свадьба' in lemmas:
        return 5
    
#вызов функции purpose_group() и возврат группы цели в новый столбец 'purpose_group'   
df_borrower_purpose['purpose_group'] = df_borrower_purpose[0].apply(purpose_group)

['покупка', ' ', 'жилье', '\n']
['приобретение', ' ', 'автомобиль', '\n']
['дополнительный', ' ', 'образование', '\n']
['сыграть', ' ', 'свадьба', '\n']
['операция', ' ', 'с', ' ', 'жилье', '\n']
['образование', '\n']
['на', ' ', 'проведение', ' ', 'свадьба', '\n']
['покупка', ' ', 'жилье', ' ', 'для', ' ', 'семья', '\n']
['покупка', ' ', 'недвижимость', '\n']
['покупка', ' ', 'коммерческий', ' ', 'недвижимость', '\n']
['покупка', ' ', 'жилой', ' ', 'недвижимость', '\n']
['строительство', ' ', 'собственный', ' ', 'недвижимость', '\n']
['недвижимость', '\n']
['строительство', ' ', 'недвижимость', '\n']
['на', ' ', 'покупка', ' ', 'подержать', ' ', 'автомобиль', '\n']
['на', ' ', 'покупка', ' ', 'свой', ' ', 'автомобиль', '\n']
['операция', ' ', 'с', ' ', 'коммерческий', ' ', 'недвижимость', '\n']
['строительство', ' ', 'жилой', ' ', 'недвижимость', '\n']
['жилье', '\n']
['операция', ' ', 'со', ' ', 'свой', ' ', 'недвижимость', '\n']
['автомобиль', '\n']
['заниматься', ' ', 'образование'

### Вывод

Лемматизировали каждую строку списка целей для их разделения по группам. 

### Категоризация данных

In [156]:
df_borrower_purpose = df_borrower_purpose.rename(columns={0: 'purpose'})
df_borrower_purpose

,purpose,purpose_group
0,покупка жилья,1
1,приобретение автомобиля,4
2,дополнительное образование,3
3,сыграть свадьбу,5
4,операции с жильем,1
5,образование,3
6,на проведение свадьбы,5
7,покупка жилья для семьи,1
8,покупка недвижимости,2
9,покупка коммерческой недвижимости,2


### Вывод

Разделили цели кредитования на группы и выделили словарь. 

### Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [157]:
df_borrower_child = df_borrower[['children','debt']]

#функция для разделения заемщиков на 2 категории
def children_existence(children):
        if children == 0:
                return 'заемщик без детей'
        if children > 0:
                return 'заемщик с детьми'
 
df_borrower_child['children_existence'] = df_borrower_child['children'].apply(children_existence)
df_borrower_child_final = df_borrower_child.groupby('children_existence').agg({'debt': ['count', 'sum']})
#соотношение суммы к общему количеству по столбцу debt дает нам % заемщиков невернувших 
#в срок (debt=1),поэтому вычитаем полученное соотношение из 1   
df_borrower_child_final['ratio'] = (1 - (df_borrower_child_final['debt']['sum'] / df_borrower_child_final['debt']['count']))*100
df_borrower_child_final

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.


debt            ratio
                    count   sum           
children_existence                        
заемщик без детей   14149  1063  92.487102
заемщик с детьми     7376   678  90.808026

### Вывод

Погашенных кредитов в срок по заемщикам без детей больше на 2%, поэтому зависимость между наличием детей и возвратом кредита в срок есть.

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [158]:
#сохраним в переменную уникальные значения столбца "семейное положение"
family_status_unique = df_borrower['family_status'].unique()
#создание датафрейма с уникальными значениями семейного положения для финальной таблицы
df_borrower_family_status = pd.DataFrame(data=family_status_unique)
#сохраним в переменную fs 2 столбца
fs = df_borrower[['family_status','debt']]

#функция для получения соотношения вернувших кредит в срок к семейному положению
def family_status(family_status_unique):
    if family_status_unique == 'Не женат / не замужем':
        fs_no = fs[fs['family_status'] == 'Не женат / не замужем']['family_status'].count()
        fs_no_debt = fs[(fs['debt'] == 0)&(fs['family_status'] == 'Не женат / не замужем')]['debt'].count()
        ratio_fs_no_debt = fs_no_debt/fs_no * 100
        return ratio_fs_no_debt 
         
    if family_status_unique == 'женат / замужем':
        marriage_yes = fs[fs['family_status'] == 'женат / замужем']['family_status'].count()
        marriage_yes_debt = fs[(fs['debt'] == 0)&(fs['family_status'] == 'женат / замужем')]['debt'].count()
        ratio_marriage_yes_debt = marriage_yes_debt/marriage_yes * 100
        return ratio_marriage_yes_debt  
    
    if family_status_unique == 'гражданский брак':
        marriage_no = fs[fs['family_status'] == 'гражданский брак']['family_status'].count()
        marriage_no_debt = fs[(fs['debt'] == 0)&(fs['family_status'] == 'гражданский брак')]['debt'].count()
        ratio_marriage_no_debt = marriage_no_debt/marriage_no * 100
        return ratio_marriage_no_debt
    
    if family_status_unique == 'вдовец / вдова':
        fs_v = fs[fs['family_status'] == 'вдовец / вдова']['family_status'].count()
        fs_v_debt = fs[(fs['debt'] == 0)&(fs['family_status'] == 'вдовец / вдова')]['debt'].count()
        ratio_fs_v_debt = fs_v_debt/fs_v * 100
        return ratio_fs_v_debt 
    
    if family_status_unique == 'в разводе':
        fs_divorce = fs[fs['family_status'] == 'в разводе']['family_status'].count()
        fs_divorce_debt = fs[(fs['debt'] == 0)&(fs['family_status'] == 'в разводе')]['debt'].count()
        ratio_fs_divorce_debt = fs_divorce_debt/fs_divorce * 100
        return ratio_fs_divorce_debt     
       
df_borrower_family_status['ratio'] = df_borrower_family_status[0].apply(family_status)
df_borrower_family_status = df_borrower_family_status.rename(columns={0: 'family_status'})
df_borrower_family_status

,family_status,ratio
0,женат / замужем,92.479806
1,гражданский брак,90.711037
2,Не женат / не замужем,90.259509
3,вдовец / вдова,93.437500
4,в разводе,92.887029


### Вывод



 % возврата кредита неодинаков по заемщикам с разным семейным положением, зависимость между семейным положением и погашением в срок есть. 

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [33]:
#создаем датафрейм с значениями из цикла по заполнению пропусков
df_borrower_income = pd.DataFrame(data=ratio_list)
df_borrower_income1 = pd.DataFrame(data=median_list)
df_borrower_income['total_income'] = df_borrower_income1[0]
df_borrower_income['income_type'] = df_borrower['income_type'].sort_values().unique()
df_borrower_income = df_borrower_income.rename(columns={0: 'ratio', '': 'total_income'})
df_borrower_income[['income_type','total_income','ratio']]   

,income_type,total_income,ratio
0,безработный,131339.751676,50.000000
1,в декрете,53829.130729,0.000000
2,госслужащий,139034.452386,94.105552
3,компаньон,162401.351555,92.605703
4,пенсионер,110179.955936,94.398340
5,предприниматель,374372.358710,100.000000
6,сотрудник,133546.457238,90.457775
7,студент,98201.625314,100.000000


### Вывод

Возврат кредита в срок зависит от типа занятости, т.е. от уровня дохода.

- Как разные цели кредита влияют на его возврат в срок?

In [152]:
#объединим основной датасет со словарем "df_borrower_purpose"
df_borrower = df_borrower.merge(df_borrower_purpose)
df_borrower_purpose_final = df_borrower.pivot_table(['debt'], ['purpose_group'])
df_borrower_purpose_final['ratio'] = (1 - df_borrower_purpose_final['debt'])*100
df_borrower_purpose_final


if 'жилье' in lemmas:
        return 1     
    if 'недвижимость' in lemmas:
        return 2
    if 'образование' in lemmas:
        return 3
    if 'автомобиль' in lemmas:
        return 4
    if 'свадьба' in lemmas:
        return 5

,debt,ratio
purpose_group,,
1,0.068858,93.114241
2,0.074446,92.555364
3,0.091994,90.800597
4,0.093395,90.660487
5,0.079216,92.078365


### Вывод

По ипотечным кредитам (группа 1 и 2) % возврата в срок выше, чем по кредитам на автомобиль, оброзование или свадьбу.

### Шаг 4. Общий вывод

На платежеспособность клиента влияют все факторы, которые мы рассмиотрели и наличие детей, и семейное положение. Поэтому данный анализ можно использовать для оценки платежеспособности потенциального клиента.